In [7]:
import os 


In [8]:
%pwd

'/Users/ravina/Desktop/project_mlflow/research'

from above we cn see that we are in the reasearch folder, but i want to go back to the folder /Users/ravina/Desktop/project_mlflow
so that i can do everything in the project folder.
so to back to one folder back following is the required command


In [9]:
os.chdir("../")

In [10]:
%pwd

'/Users/ravina/Desktop/project_mlflow'

In [6]:
#entity: return type of a function
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)  #this is not python class but dataclass, here you can define the veriables without using self keyword.
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path



In [7]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories




In [8]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.get('data_ingestion')
        if config is None:
            raise ValueError("Data Ingestion configuration is missing or invalid.")

        create_directories([config.get('root_dir')])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [14]:
import os 
import urllib.request as request
import zipfile
from mlproject import logger
from mlproject.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-10 22:56:36,693:INFO:yaml file:config/config.yaml loaded successfully]
[2024-01-10 22:56:36,696:INFO:yaml file:params.yaml loaded successfully]
[2024-01-10 22:56:36,698:INFO:yaml file:Schema.yaml loaded successfully]
[2024-01-10 22:56:36,699:INFO:created directory at:artifacts]
[2024-01-10 22:56:36,699:INFO:created directory at:artifacts/data_ingestion]
[2024-01-10 22:56:36,700:INFO:File already exists of size: ~22 KB]
